# All endpoints for model v1

## Imports

In [1]:
import json
import os
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
READER_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

# Load reader model
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0


## Fake functions using BDD for tests

In [3]:
# In this example the 3 questions were not from the same subcategory
# I let you write this function MAGB

def get_random_questions_from_subcategory(subcategory, type, k=3):
    """ 
    Get k random questions verified from subcategory from a specific type in our database.

    Parameters:
    subcatecory (str): Subcategory name. 
    type (int): Type is MCQ or Open question.
    k (int): Number of questions to return.

    Returns:
    str: The k questions.
    """

    if type == 'MCQ':
        pass
    elif type == 'Open':
        # 2024 EPAC Open the first 3
        q1 = " International application WO -X was filed at the EPO on 27 August 2024. No fees have been paid.  1. What fees are due on filing for WO -X? Fee amounts need not be mentioned. 2. What is the time limit for paying these fees? 3. What happens if these fees are not paid within the time limit, and what can you do about it?"
        q2 = " On 25  October 2019, the Spanish University Isabel  II and the company Tomato Matters filed a European patent application in Spanish, accompanied by a translation into English. Tomato Matters employs more than 260 employees.  The University Isabel  II has filed two patent applications with the EPO over the past five years. On 10 October 2024, Tomato Matters transfers its rights to Naranjas Navel , a company which employs 9 members of staff and whose annual turnover is EUR  1 million. Naranjas Navel has never filed any patent applications with the EPO.  In a communication from the EPO under Rule 71(3) EPC dated 10 October 2024, the name of the applicants is given as: Isabe  III (clerical error) and Tomato Matters.  1. What has to be done to obtain a Unitary Patent as soon as possible for Isabel  II and Naranjas Navel? Is it possible to benefit from the compensation scheme?  Please list the necessary steps at minimum cost. You should identify the fees that have to be paid, but you do not need to specify their amounts.  2. Let us now suppose that the request for unitary effect has been refused. What is the time limit for lodging  an application to reverse this  decision, and to whom should the application be addressed? "
        q3 = " In March 2018, a European patent application was filed in French. A European patent was granted in June 2023. Unitary effect has been registered and the proprietor has filed a statement concerning licences of right. The patent has also been validated in Sp ain and in Croatia; the European patent is still in force in these states.  The proprietor filed a request for limitation of the patent. The examining division has issued an interlocutory decision, indicating that the patent with amended claims and an amended description meets the requirements of the EPC. The mention of the limita tion will be published in the last European Patent Bulletin of 2024.  1. To maintain the existing patents, what translations must be filed, at which offices? 2. Do any fees have to be paid? Fee amounts need not be mentioned. "
    
    print(f'{q1}\n{q2}\n{q3}')
    return f'{q1}\n{q2}\n{q3}'

## *get_context*

In [4]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
def get_context(query, k=5):
    """ 
    Retrieves relevant context for a given query.

    Parameters:
    query (str): The input query for which context is needed.
    k (int, optional): The number of relevant context elements to retrieve (default is 5).

    Returns:
    list: A list containing relevant context elements.
    """

    # Load embeddings
    embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        multi_process=True,
        model_kwargs={"device": "cpu"},  # replace 'cpu' by 'cuda' if you have Nvidia gpu
        encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
    )
    KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local("../outputs/rag_embeddings_thenlper_gte-small", embedding_model, allow_dangerous_deserialization=True)

    # Retrieve docs
    # print(f"\nStarting retrieval for {query=}...")
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=query, k=k)

    return retrieved_docs

## *generate_mcq*  (Not working)

- Réecrire prompt
- Essayer de contraindre sortie en json

In [5]:
def generate_mcq(subcategory):
    """
    Generates an MCQ question.

    Parameters:
    subcatecory (str): Subcategory name.

    Returns:
    question (dict): {'question': '...',
                      'options': ['A ....', 'B ...', ...]}
    """

    # Build prompt
    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """You are given a list of questions and a context extracted from those questions. Your task is to generate a new question that matches the style and structure of the given questions while ensuring that it is correct and relevant to the provided context.  

            **Instructions:**
            - The new question must be coherent with the other questions in terms of phrasing, complexity, and format.
            - Ensure that the new question can be answered using the provided context.
            - Do not introduce any information that is not supported by the context.
            - If the context does not allow for a new relevant question, state: "A new relevant question cannot be generated based on the given context."
            - Output **only** the new question, with no explanations, formatting, or additional text.
            """,
        },
        {
            "role": "user",
            "content": """Context:  
        {context}  
        ---  
        Given the following questions:  
        {existing_questions}  

        Generate a new question that follows the same format and is correct based on the context.""",
        }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)
    
    # Get questions
    questions = get_random_questions_from_subcategory(subcategory=subcategory, type='MCQ', k=3)

    # Retrieve context
    retrieved_docs = get_context(questions, k=5)
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content} \nSource: {doc.metadata['ref']}\n\n' for i, doc in enumerate(retrieved_docs)])
    context_sources = "".join([f'\nSource: {doc.metadata['ref']}, Url: {doc.metadata.get('url', 'N/A')}' for i, doc in enumerate(retrieved_docs)])

    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(existing_questions=questions, context=context)
    
    # Redact an answer
    new_question = READER_LLM(final_prompt)[0]["generated_text"]

    return new_question, questions, context_sources

## *generate_mcq_answer* (Not working)

## *generate_open*

In [6]:
def generate_open(subcategory):
    """
    Generates an Open question.

    Parameters:
    subcatecory (str): Subcategory name.

    Returns:
    question (str): The new question.
    """
    
    # Build prompt
    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """You are given a list of questions and a context extracted from those questions. Your task is to generate a new question that matches the style and structure of the given questions while ensuring that it is correct and relevant to the provided context.  

            **Instructions:**
            - The new question must be coherent with the other questions in terms of phrasing, complexity, and format.
            - Ensure that the new question can be answered using the provided context.
            - Do not introduce any information that is not supported by the context.
            - If the context does not allow for a new relevant question, state: "A new relevant question cannot be generated based on the given context."
            - Output **only** the new question, with no explanations, formatting, or additional text.
            """,
        },
        {
            "role": "user",
            "content": """Context:  
        {context}  
        ---  
        Given the following questions:  
        {existing_questions}  

        Generate a new question that follows the same format and is correct based on the context.""",
        }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)
    
    # Get questions
    questions = get_random_questions_from_subcategory(subcategory=subcategory, type='Open', k=3)

    # Retrieve context
    retrieved_docs = get_context(questions, k=5)
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content} \nSource: {doc.metadata['ref']}\n\n' for i, doc in enumerate(retrieved_docs)])
    # context_sources = "".join([f'\nSource: {doc.metadata['ref']}, Url: {doc.metadata.get('url', 'N/A')}' for i, doc in enumerate(retrieved_docs)])

    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(existing_questions=questions, context=context)
    
    # Redact an answer
    new_question = READER_LLM(final_prompt)[0]["generated_text"]

    return new_question

## *generate_open_answer*

In [7]:
def generate_open_answer(question):
    """
    Generates an answer to a given open question.

    Parameters:
    question (str): The input question for which an answer is needed.

    Returns:
    answer (str): The generated response from the AI with the context used.
    """

    # Build prompt
    prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Use only the information contained in the provided context to generate a precise and relevant answer to the given question.  
        
        **General Rules:**
        - Answer concisely and directly to the question.  
        - If the answer requires choosing from multiple options, explicitly state the correct answer.  
        - Always cite the sources used and explain their relevance to the answer.  
        - If the answer cannot be deduced from the context, explicitly state that it cannot be answered.  

        **For Multiple Choice Questions (MCQs):**  
        - Start your response with: **"The correct answer is: [option]"** (e.g., "The correct answer is: A.")  
        - Explain why this option is correct based on the provided context.  
        - Briefly justify why the other options are incorrect, if possible.  
        
        **Example of response format:**  
        - **The correct answer is: [option]**  
        - **Justification:** (Explain why this answer is correct, citing sources)  
        - **Why other options are incorrect:** (Briefly explain why the other options do not apply)  

        If the question is not a multiple-choice question, provide a direct and structured answer.  
        """,
    },
    {
        "role": "user",
        "content": """Context:  
    {context}  
    ---  
    Now, answer the following question.  

    **Question:** {question}""",
    }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)
    

    # Retrieve context
    retrieved_docs = get_context(question)
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content}\nSource: {doc.metadata['ref']}\n' for i, doc in enumerate(retrieved_docs)])
    context_sources = "".join([f'\nSource: {doc.metadata['ref']}, Url: {doc.metadata.get('url', 'N/A')}' for i, doc in enumerate(retrieved_docs)])
    
    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)
    
    # Redact an answer
    answer = READER_LLM(final_prompt)[0]["generated_text"]

    # Assemble answer and context_sources
    final_answer = f'Answer:\n{answer}\n\nSources:{context_sources}'

    return final_answer

## *generate_feedback*

In [8]:
def generate_feedback(ai_question, ai_answer, user_answer):
    """
    Generates an AI-generated feedback on the user_answer. 
    The ai_question and ai_answer were generated before. When we gave an ai_question to an user,
    we also take the ai_answer. So when the user answer, we can give all, ai_question, ai_answer (the correct one), 
    and user_answer to give a feedback to the user.

    Parameters:
    ai_question (str): The question generated by AI.
    ai_answer (str): The correct answer.
    user_answer (str): The user answer.

    Returns:
    feedback (str): The correct answer and the explaination why the user is wrong including the context.
    """

    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """You are given a question, a correct answer, and a user-provided answer, along with relevant context. Your task is to evaluate the user's answer and provide feedback explaining any errors.  

            **Instructions:**
            - Compare the user's answer with the correct answer and identify any inaccuracies.
            - If the user uses incorrect references, explain why they are wrong and provide the correct information.
            - Highlight any misunderstandings or misinterpretations the user may have made.
            - If the user's answer is correct, acknowledge it and confirm why it is valid.
            - Ensure your feedback is clear, concise, and instructional.
            """,
        },
        {
            "role": "user",
            "content": """Context:  
        {context}  
        ---  
        **Question:** {ai_question}  
        **Correct Answer:** {ai_answer}  
        **User's Answer:** {user_answer}  

        Provide feedback explaining any errors in the user's answer and clarify why it is incorrect, referencing the provided context where necessary.""",
        }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)

    # Retrieve context
    ai_question_context = get_context(ai_question, k=3)
    ai_answer_context = get_context(ai_answer, k=3)
    user_answer_context = get_context(user_answer, k=3)
    # Combine all retrieved contexts
    all_contexts = ai_question_context + ai_answer_context + user_answer_context
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content} \nSource: {doc.metadata['ref']}\n\n' for i, doc in enumerate(all_contexts)])
    context_sources = "".join([f'\nSource: {doc.metadata['ref']}, Url: {doc.metadata.get('url', 'N/A')}' for i, doc in enumerate(all_contexts)])
    
    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(ai_question=ai_question, ai_answer=ai_answer, user_answer=user_answer, context=context)
    
    # Redact an answer
    feedback = READER_LLM(final_prompt)[0]["generated_text"]

    # Assemble final answer
    final_answer = f'{ai_answer}\n\nFeedback:\n{feedback}\n\nContext:{context_sources}'

    return final_answer

## *chat_with_ai*

In [9]:
def chat_with_ai(conversation_history, user_message):
    """
    Based on the history of the conversation, initialy filled with question, user_answer, feedback.

    Parameters:
    conversation_history (str): Initialy the quesiton, user_answer, feedback. The history is filled with new messages.
    user_message (str): New message from user.

    Returns:
    answer (str): The answer for the user_message, base on the context from history.
    context_sources (str): The context used to answer with real link.
    """

    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """You are an AI assistant engaged in an ongoing conversation with a user. Your task is to generate a relevant and coherent response based on the conversation history and extracted context.  

            **Instructions:**  
            - Analyze the provided conversation history to understand the discussion.  
            - Use the extracted context to ensure factual accuracy and relevance in your response.  
            - Maintain the same tone, style, and level of detail as previous responses.  
            - Do not introduce any information that is not supported by the context or history.  
            - If the context does not provide sufficient information, respond naturally while acknowledging the limitation.  
            - Ensure your response is clear, concise, and helpful.  
            """,
        },
        {
            "role": "user",
            "content": """Conversation History:  
        {conversation_history}  
        ---  
        Extracted Context:  
        {context}  
        ---  
        User Message: {user_message}  

        Generate a response that continues the conversation naturally while ensuring accuracy based on the provided context.""",
        }
    ]

    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(prompt_in_chat_format, tokenize=False, add_generation_prompt=True)

    # Retrieve context
    conversation_history_context = get_context(conversation_history, k=5)
    user_message_context = get_context(user_message, k=3)
    # Combine all retrieved contexts
    all_contexts = conversation_history_context + user_message_context
    context = "\nExtracted documents:\n"
    context += "".join([f'Content: {doc.page_content} \nSource: {doc.metadata['ref']}\n\n' for i, doc in enumerate(all_contexts)])
    context_sources = "".join([f'\nSource: {doc.metadata['ref']}, Url: {doc.metadata.get('url', 'N/A')}' for i, doc in enumerate(all_contexts)])
    
    # Add context to prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(conversation_history=conversation_history, user_message=user_message, context=context)
    
    # Redact an answer
    answer = READER_LLM(final_prompt)[0]["generated_text"]

    # Assemble answer
    final_answer = f'{answer}\n\nContext:\n{context_sources}'

    return final_answer

# Exemple of an exchange: 

## MCQ

## Open

In [10]:
# Initialize history
history = ''

In [11]:
# Generate a new question
question = generate_open(subcategory='Subcategory')
history += f'Question:\n{question}'
print(question)

 International application WO -X was filed at the EPO on 27 August 2024. No fees have been paid.  1. What fees are due on filing for WO -X? Fee amounts need not be mentioned. 2. What is the time limit for paying these fees? 3. What happens if these fees are not paid within the time limit, and what can you do about it?
 On 25  October 2019, the Spanish University Isabel  II and the company Tomato Matters filed a European patent application in Spanish, accompanied by a translation into English. Tomato Matters employs more than 260 employees.  The University Isabel  II has filed two patent applications with the EPO over the past five years. On 10 October 2024, Tomato Matters transfers its rights to Naranjas Navel , a company which employs 9 members of staff and whose annual turnover is EUR  1 million. Naranjas Navel has never filed any patent applications with the EPO.  In a communication from the EPO under Rule 71(3) EPC dated 10 October 2024, the name of the applicants is given as: Isab

C:\Users\Patrice\AppData\Local\Temp\ipykernel_10804\2683960810.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In January 2022, a European patent application was filed in Portuguese. The applicant is a small business employing fewer than 10 employees. The application includes a set of claims in English. How many days does the applicant have to file a translation into one of the official languages of the European Patent Office, and what penalty might result from failing to do so?

---

This new question maintains the same format and complexity as the original questions while adhering to the provided context. It asks about the deadline for filing a translation into one of the official languages of the European Patent Office when a European patent application is filed in a non-official language, which aligns with the content related to translations in the given context.


In [ ]:
# Generate ai_answer (the correct one)
correct_answer = generate_open_answer(question)

Device set to use cuda:0



Starting retrieval for query='In January 2022, a European patent application was filed in Japanese. The applicant is a Japanese citizen residing in Japan. According to Article 14(2), a 30% reduction of the filing and/or examination fee for certain categories of applicants is provided for. When should the applicant file the documents making up the application "as filed" and/or the request for examination in an admissible non-EPO language, and when should the translation be filed?'...


In [ ]:
# The user answer
user_answer = """I have no idea bro !"""
history += f'\n\nUser Answer:\n{user_answer}'

In [ ]:
# Generat AI feedback
feed_back = generate_feedback(question, correct_answer, user_answer)
history += f'\n\nFeedback:\n{feed_back}'
print(feed_back)

Device set to use cuda:0



Starting retrieval for query='In January 2022, a European patent application was filed in Japanese. The applicant is a Japanese citizen residing in Japan. According to Article 14(2), a 30% reduction of the filing and/or examination fee for certain categories of applicants is provided for. When should the applicant file the documents making up the application "as filed" and/or the request for examination in an admissible non-EPO language, and when should the translation be filed?'...

Starting retrieval for query='Answer:\nThe applicant should file the documents making up the application "as filed" and/or the request for examination in an admissible non-EPO language within the time limits specified in Article 14(2) of the EPC. Specifically:\n\n- Documents making up the application "as filed": Within two months of filing the European patent application.\n\n- Request for examination in an admissible non-EPO language: Within one month of filing the document.\n\nAdditionally, according to 

In [32]:
# Open chat
user_message = "Explain me with more details and references."
chat_answer = chat_with_ai(history, user_message)
history += f'\n\nUser message:\n{user_message}'
history += f'\n\nChat answer:\n{chat_answer}'
print(chat_answer)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Device set to use cuda:0


ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
print(history)

Question:
In January 2022, a European patent application was filed in Japanese. The applicant is a Japanese citizen residing in Japan. According to Article 14(2), a 30% reduction of the filing and/or examination fee for certain categories of applicants is provided for. When should the applicant file the documents making up the application "as filed" and/or the request for examination in an admissible non-EPO language, and when should the translation be filed?

User Answer:
 
I have no idea bro !


Feedback:
Correct answer:
Answer:
The applicant should file the documents making up the application "as filed" and/or the request for examination in an admissible non-EPO language within the time limits specified in Article 14(2) of the EPC. Specifically:

- Documents making up the application "as filed": Within two months of filing the European patent application.

- Request for examination in an admissible non-EPO language: Within one month of filing the document.

Additionally, according t